本周只有一个代码实践题

### <评阅点>     
1. 完成代码并无bug  50‘   
2. 代码有部分bug   40'   
3. 代码有重大bug   30‘
4. 代码不完整  20'   
5. 其余 0‘   

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [1]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from torch.utils.data import DataLoader,Dataset

/Users/wangqin/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/wangqin/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/wangqin/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/wangqin/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/wangqin/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may 

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('../13th/small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
# model.train()
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

In [4]:
def top_n_filtering(logits, top_n = 10, filter_value = -float('Inf')):
    assert logits.dim() == 1
    sorted_logits, sorted_indices = torch.sort(logits, descending = True)

    sorted_indices_to_remove = sorted_indices >= top_n
    sorted_indices_to_remove[..., :top_n] = 0 # set top n probabilities as 0
    sorted_indices_to_remove[..., top_n:] = 1

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = filter_value
    return logits 

In [5]:
def recalc():
    global conditioned_tokens
    global generated_tokens
    
    indexed_tokens = conditioned_tokens + generated_tokens

    tokens_tensor = torch.tensor([indexed_tokens]) # 
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    logits = predictions[0, -1, :]

    filtered_logits = top_n_filtering(logits)

    probabilities = F.softmax(filtered_logits, dim = -1)

    next_token = torch.multinomial(probabilities, 1)

    generated_tokens.append(next_token.item())
    return next_token.item() 

In [6]:
def generate():
    global conditioned_tokens
    global generated_tokens
    
    if len(tokenizer.decode(conditioned_tokens)) > 320: # 320 is a number that can be set based on your needs
        dc = tokenizer.decode(conditioned_tokens)
        dc = dc[len(dc) - 320:]
        idx = dc.find('<|endoftext|>')
        if idx != -1:
            dc = dc[idx + len('<|endoftext|>'):]
            
        conditioned_tokens = tokenizer.encode(dc)
        
    while True:
        result = recalc()
        
        if result == 50256:
            # end of text: 50256
            
            decoded_reply = tokenizer.decode(generated_tokens)
            
            to_print = decoded_reply
            if to_print.endswith('<|endoftext|>'):
                to_print = to_print[:-len('<|endoftext|>')]
            print(to_print)
            
            conditioned_tokens += (tokenizer.encode(decoded_reply))
            
            generated_tokens = []
            
            break 

In [25]:
conditioned_tokens = []
generated_tokens = []
## k-sampling
first_text = "What is the meaning of life?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')

K-sampling:

I'm sorry, what's that supposed to mean?
#################



In [28]:
conditioned_tokens = []
generated_tokens = []

first_text = "Does money buy happiness?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')

K-sampling:

But money can't buy happiness.
#################



In [32]:
conditioned_tokens = []
generated_tokens = []

first_text = "what is the meaning of a good life?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')



K-sampling:

It has the meaning of life, but the meaning is not the same
#################



In [41]:
conditioned_tokens = []
generated_tokens = []

first_text = "How to be a good person?"
conditioned_tokens += tokenizer.encode('\t' + first_text) + [50256]
print('K-sampling:\n')
generate()
print('#################\n')

K-sampling:

I think you're confusing the words to be a good person with the words to be a good person in the past...
#################



### Summary

The output for each question is not so good, maybe it is due to the use of small GPT2 model. For better performance, medium or large model is neccessary.